In [1]:
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import os
import numpy as np
import cv2 as cv
import re
import pandas as pd

In [2]:
def extract_position(input_name):
    # Use regex to find the 'Position' part
    match1 = re.search(r'(Position \d+ - \d+_)(Z\d+_C\d+)', input_name)
    match2 = re.search(r'(Position \d+_)(Z\d+_C\d+)', input_name)
    if match1:
        return match1.group(0), match1.group(1), match1.group(2)
    elif match2:
        return match2.group(0), match2.group(1), match2.group(2)
    else:
        return "Unknown_Position"

In [3]:
folder2 = 'Images from Dellaire Lab/TIFF SAEC rep1 63x no As/pml_transformed'

In [4]:
import os
import re
import cv2 as cv
import numpy as np
from pathlib import Path

def extract_position(input_name):
    """Extract position and projection information from filename."""
    # Use regex to find the 'Position' part
    match1 = re.search(r'(Position \d+ - \d+_)(Z\d+_C\d+)', input_name)
    match2 = re.search(r'(Position \d+_)(Z\d+_C\d+)', input_name)
    if match1:
        return match1.group(0), match1.group(1), match1.group(2)
    elif match2:
        return match2.group(0), match2.group(1), match2.group(2)
    else:
        return "Unknown_Position", "Unknown_Position", "Unknown_Position"

def merge_positions(input_folder, output_folder, file_prefix="transformed_pml"):
    """
    Merge images based on their positions using weighted averaging.
    
    Parameters:
    -----------
    input_folder : str
        Path to folder containing input images
    output_folder : str
        Path to save merged images
    file_prefix : str
        Prefix of the input files
    """
    # Create output folder if it doesn't exist
    Path(output_folder).mkdir(parents=True, exist_ok=True)
    
    # Get and sort image files
    images = sorted([f for f in os.listdir(input_folder) if f.startswith(file_prefix)])
    
    if not images:
        raise ValueError(f"No images found with prefix '{file_prefix}' in {input_folder}")
    
    # Extract position information
    image_names = []
    positions = []
    projections = []
    
    for img in images:
        val1, val2, val3 = extract_position(img)
        positions.append(val2)
        projections.append(val3)
        image_names.append(val1)
    
    # Get unique sorted positions and projections
    positions = sorted(list(set(positions)))
    projections = sorted(list(set(projections)))
    
    print(f"Found {len(positions)} positions and {len(projections)} projections")
    
    # Read first image to get dimensions
    first_img_path = os.path.join(input_folder, images[0])
    template_image = cv.imread(first_img_path)
    
    if template_image is None:
        raise ValueError(f"Could not read first image: {first_img_path}")
    
    # Process each position
    for position in positions:
        # Initialize empty image
        merged_image = np.zeros(template_image.shape, template_image.dtype)
        count = 0
        
        for projection in projections:
            # Construct filename
            img_path = os.path.join(
                input_folder, 
                f"{file_prefix} - {position}{projection}.jpg"
            )
            
            if os.path.exists(img_path):
                image = cv.imread(img_path)
                
                if image is not None and image.shape == merged_image.shape:
                    # Add weighted images
                    merged_image = cv.addWeighted(merged_image, 0.5, image, 0.5, 0)
                    count += 1
                else:
                    print(f"Warning: Skipping {img_path} due to invalid image or shape mismatch")
        
        if count > 0:
            # Save merged image
            output_path = os.path.join(output_folder, f"merged - {position}.jpg")
            cv.imwrite(output_path, merged_image)
            print(f"Merged {count} projections for {position}")
        else:
            print(f"Warning: No valid images found for {position}")

# Example usage
if __name__ == "__main__":
    input_folder = "Images from Dellaire Lab/TIFF SAEC rep1 63x no As/pml_transformed"
    output_folder = "Images from Dellaire Lab/TIFF SAEC rep1 63x no As/merged_projections_pml"
    
    try:
        merge_positions(input_folder, output_folder)
    except Exception as e:
        print(f"Error: {str(e)}")

['transformed_pml - Position 1 - 1_Z00_C1.jpg', 'transformed_pml - Position 1 - 1_Z01_C1.jpg', 'transformed_pml - Position 1 - 1_Z02_C1.jpg', 'transformed_pml - Position 1 - 1_Z03_C1.jpg', 'transformed_pml - Position 1 - 1_Z04_C1.jpg', 'transformed_pml - Position 1 - 1_Z05_C1.jpg', 'transformed_pml - Position 1 - 1_Z06_C1.jpg', 'transformed_pml - Position 1 - 1_Z07_C1.jpg', 'transformed_pml - Position 1 - 1_Z08_C1.jpg', 'transformed_pml - Position 1 - 1_Z09_C1.jpg', 'transformed_pml - Position 1 - 1_Z10_C1.jpg', 'transformed_pml - Position 1 - 1_Z11_C1.jpg', 'transformed_pml - Position 1 - 1_Z12_C1.jpg', 'transformed_pml - Position 1 - 1_Z13_C1.jpg', 'transformed_pml - Position 1 - 1_Z14_C1.jpg', 'transformed_pml - Position 1 - 1_Z15_C1.jpg', 'transformed_pml - Position 1 - 1_Z16_C1.jpg', 'transformed_pml - Position 1 - 1_Z17_C1.jpg', 'transformed_pml - Position 1 - 1_Z18_C1.jpg', 'transformed_pml - Position 1 - 1_Z19_C1.jpg', 'transformed_pml - Position 1 - 1_Z20_C1.jpg', 'transformed